# Aspirational DistilBERT Model Using Merged Data Batch 1 + Batch 2 Experiment 2

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
merged_aspirational_df_batch_1 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Aspirational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')
merged_aspirational_df_batch_2 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/Aspirational Plus_sentence_level_batch_2_jaccard.csv", encoding='utf-8')

merged_aspirational_df = pd.concat([merged_aspirational_df_batch_1, merged_aspirational_df_batch_2])

# Shuffle the merged dataset
merged_aspirational_df = shuffle(merged_aspirational_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_aspirational_df, test_size=0.1, random_state=42, stratify=merged_aspirational_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [5]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (8856, 3) 
Test dataset shape: (985, 3)
Positive labels present in the dataset : 805  out of 8856 or 9.08988256549232%
Positive labels present in the test dataset : 89  out of 985 or 9.035532994923857%


In [7]:
print(training_df.shape)
print(test_df.shape)

(8856, 3)
(985, 3)


## 2. Experimental Design

In [8]:
MAXLEN = 150

In [9]:
X = training_df['sentence']
y = training_df['label']

In [10]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)
X_test.shape

(886,)

In [11]:
model_name = 'bert-base-uncased'

In [12]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 58


In [13]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.54999379 5.50062112]


In [14]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=16)
# learner.fit_onecycle(2e-5, 4, class_weight=class_weights)
# learner.autofit(2.27E-06, early_stopping=4)
# distillbert_learner.set_weight_decay(0.001)
distillbert_learner.autofit(2.06E-05, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2.06e-05...
Epoch 1/1024
499/499 [==============================] - 310s 594ms/step - loss: 0.5698 - accuracy: 0.6767 - val_loss: 0.5986 - val_accuracy: 0.6140
Epoch 2/1024
499/499 [==============================] - 289s 577ms/step - loss: 0.4439 - accuracy: 0.7248 - val_loss: 0.4863 - val_accuracy: 0.7111
Epoch 3/1024
499/499 [==============================] - 288s 575ms/step - loss: 0.3841 - accuracy: 0.7770 - val_loss: 0.5744 - val_accuracy: 0.7020
Epoch 4/1024
499/499 [==============================] - 302s 603ms/step - loss: 0.3261 - accuracy: 0.8211 - val_loss: 0.4767 - val_accuracy: 0.7731
Epoch 5/1024
499/499 [==============================] - 294s 587ms/step - loss: 0.2813 - accuracy: 0.8482 - val_loss: 0.4845 - val_accuracy: 0.7799
Epoch 6/1024
499/499 [==============================] - 297s 595ms/step - loss: 0.2403 - accuracy: 0.8708 - val_loss: 0.4019

In [15]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

28/28 [==============================] - 15s 412ms/step
              precision    recall  f1-score   support

           0       0.97      0.84      0.90       805
           1       0.31      0.70      0.43        81

    accuracy                           0.83       886
   macro avg       0.64      0.77      0.67       886
weighted avg       0.91      0.83      0.86       886



array([[680, 125],
       [ 24,  57]])

In [16]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [18]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [19]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [20]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [21]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 760, False Positive: 136, False Negative: 34, True Positive: 55


In [22]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.85      0.90       896
           1       0.29      0.62      0.39        89

    accuracy                           0.83       985
   macro avg       0.62      0.73      0.65       985
weighted avg       0.90      0.83      0.85       985
 



In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [21]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.6427102188579892
